In [1]:
from modules.utils import load_yaml, save_yaml, get_logger

from modules.earlystoppers import EarlyStopper
from modules.recorders import Recorder
from modules.datasets import QADataset
from modules.trainer import Trainer

from modules.optimizers import get_optimizer
from modules.metrics import get_metric
from modules.losses import get_loss
from models.utils import get_model

from transformers import ElectraTokenizerFast, ElectraTokenizer
from transformers import AutoTokenizer

from torch.utils.data import DataLoader
import torch

from datetime import datetime, timezone, timedelta
import numpy as np
import random
import os
import copy

import wandb

In [2]:
# Root directory
PROJECT_DIR = os.path.dirname('.')

# Load config
config_path = os.path.join(PROJECT_DIR, 'config', 'train_config.yml')
config = load_yaml(config_path)

# Train Serial
kst = timezone(timedelta(hours=9))
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# Recorder directory
DEBUG = config['TRAINER']['debug']
print(f'debug {DEBUG}')

RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
os.makedirs(RECORDER_DIR, exist_ok=True)

# Data directory
DATA_DIR = config['DIRECTORY']['dataset']

# Seed
torch.manual_seed(config['TRAINER']['seed'])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(config['TRAINER']['seed'])
random.seed(config['TRAINER']['seed'])

# GPU
os.environ['CUDA_VISIBLE_DEVICES'] = str(config['TRAINER']['gpu'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



debug True


In [3]:
"""
00. Set Logger
"""
logger = get_logger(name='train', dir_=RECORDER_DIR, stream=False)
logger.info(f"Set Logger {RECORDER_DIR}")
print('/'.join(logger.handlers[0].baseFilename.split('/')[:-1]))

/home/kotech/workspace/webstudy/deep/chal2022/results/train/20220612_020902


In [4]:
"""
01. Load data
"""
# Load tokenizer

tokenizer_dict = {'ElectraTokenizerFast': ElectraTokenizerFast, 'ElectraTokenizer': ElectraTokenizer }

#tokenizer = tokenizer_dict[config['TRAINER']['tokenizer']].from_pretrained(config['TRAINER']['pretrained'])
tokenizer = AutoTokenizer.from_pretrained(config['TRAINER']['pretrained'])
print(config['TRAINER']['tokenizer'])
print(config['TRAINER']['pretrained'])
print(tokenizer)

ElectraTokenizerFast
monologg/koelectra-base-v3-discriminator
PreTrainedTokenizerFast(name_or_path='monologg/koelectra-base-v3-discriminator', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [5]:
import json
def load_json(path):
    with open(path, 'r', encoding='utf8') as f:
        return json.load(f)

In [6]:
data = load_json('dataset/train.json')

In [5]:
def read_squad(mode, data, debug=False):
    contexts = []
    questions = []
    question_ids = []
    answers = []
    data = data.copy()

    # train - val split
    if mode == 'train':
        data['data'] = data['data'][:-1*int(len(data['data'])*0.1)]
    elif mode == 'val':
        data['data'] = data['data'][-1*int(len(data['data'])*0.1):]


    till = 100 if debug else len(data['data'])


    for group in data['data'][:till]:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if mode == 'test':
                    contexts.append(context)
                    questions.append(question)
                    question_ids.append(qa['question_id'])
                else: # train or val
                    for ans in qa['answers']:
                        contexts.append(context)
                        questions.append(question)

                        if qa['is_impossible']:
                            answers.append({'text':[''],'answer_start':[-1]})
                        else:
                            #answers.append([ans])
                            answers.append({'text':[ans['text']],'answer_start':[ans['answer_start']]})
    # return formatted data lists
    return contexts, questions, answers, question_ids

In [6]:
from datasets import Dataset
#import pandas as pd
contexts, questions, answers, question_ids = read_squad('train', data)
ids = []
for i in range(len(contexts)):
    ids.append(i)
train_dataset = Dataset.from_dict({'id':ids, 'context': contexts, 'question':questions,
                                   'answers':answers})
    
#df = pd.DataFrame({'context': contexts, 'question': questions, 'answers': answers})
#df.to_csv('train.csv', index=False)
contexts, questions, answers, question_ids = read_squad('val', data)
ids = []
for i in range(len(contexts)):
    ids.append(i)
val_dataset = Dataset.from_dict({'id':ids, 'context': contexts, 'question':questions,
                                   'answers':answers})
#df = pd.DataFrame({'context': contexts, 'question': questions, 'answers': answers})
#df.to_csv('val.csv', index=False)

In [7]:
from datasets import DatasetDict
datasets = DatasetDict({'train':train_dataset, 'validation':val_dataset})

In [8]:
train_dataset[0]

{'id': 0,
 'context': '이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재의 경쟁력보다는 미래의 경쟁력을 평가하도록 평가의 대상을 확대해야 한다. 지금까지의 경쟁력 평가가 현위치를 중심으로 한 것이었다면, 앞으로의 대안은 과정과 경로를 중시하는 것이 되어야 한다는 것이다. 사실 지식정보화사회에서 가진 것이란 허망한 것이다. 현재의 기술수준이 높더라고, 이를 끊임없이 개량하고 새로운 기술을 발전시킬 수 있는 능력이 없다면 미래의 전망은 어둡다. 기술이나 정보는 실물 자산보다 진부화율이 매우 높다. 둘째, 기술경쟁력 평가를 순위를 매기는 작업이 아니라 이를 통하여 장점과 단점을 파악하는 과정으로 활용하여야 한다. 순위는 대중적인 흥미는 끌 수 있지만, 그것으로부터 교훈이 도출되는 것은 아니다. 기술경쟁력의 평가를 통해서 외국시스템의 장점을 배우고, 한국 시스템의 단점을 교정하는 대안이 발견될 수 있다. 경쟁이란 남과 비교하는 것을 의미한다. 내가 아무리 잘 하더라도 남이 나보다 더 잘 한다면 경쟁에서 진다. 그래서 경쟁은 각박하지만, 과학기술은 경쟁을 피할 수 있는 분야가 아니다. 경쟁 상대국을 연구하고, 경쟁 상대국을 앞설 수 있는 방안을 강구하기 위해서 기술경쟁력에 관한 연구가 더욱 활발해져야 할 것이다.',
 'question': '경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지',
 'answers': {'answer_start': [603], 'text': ['기술경쟁력']}}

In [5]:
#max_length = 384 # The maximum length of a feature (question and context)
#doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [10]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    #tokenized_examples['input_ids'] = torch.Tensor(tokenized_examples['input_ids'])
    return tokenized_examples

In [11]:
pad_on_right = tokenizer.padding_side == "right"

In [12]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [14]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [15]:
print(tokenized_datasets['train'][0]['start_positions'])
print(tokenized_datasets['train'][0]['end_positions'])

347
350


In [16]:
tokenizer.decode(tokenized_datasets['train'][0]['input_ids'][347:350])

'기술경쟁'

In [17]:
tokenizer.decode(tokenized_datasets['train'][0]['input_ids'])

'[CLS] 경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지 [SEP] 이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재의 경쟁력보다는 미래의 경쟁력을 평가하도록 평가의 대상을 확대해야 한다. 지금까지의 경쟁력 평가가 현위치를 중심으로 한 것이었다면, 앞으로의 대안은 과정과 경로를 중시하는 것이 되어야 한다는 것이다. 사실 지식정보화사회에서 가진 것이란 허망한 것이다. 현재의 기술수준이 높더라고, 이를 끊임없이 개량하고 새로운 기술을 발전시킬 수 있는 능력이 없다면 미래의 전망은 어둡다. 기술이나 정보는 실물 자산보다 진부화율이 매우 높다. 둘째, 기술경쟁력 평가를 순위를 매기는 작업이 아니라 이를 통하여 장점과 단점을 파악하는 과정으로 활용하여야 한다. 순위는 대중적인 흥미는 끌 수 있지만, 그것으로부터 교훈이 도출되는 것은 아니다. 기술경쟁력의 평가를 통해서 외국시스템의 장점을 배우고, 한국 시스템의 단점을 교정하는 대안이 발견될 수 있다. 경쟁이란 남과 비교하는 것을 의미한다. 내가 아무리 잘 하더라도 남이 나보다 더 잘 한다면 경쟁에서 진다. 그래서 경쟁은 각박하지만, 과학기술은 경쟁을 피할 수 있는 분야가 아니다. 경쟁 상대국을 연구하고, 경쟁 상대국을 앞설 수 있는 방안을 강구하기 위해서 기술경쟁력에 관한 연구가 더욱 활발해져야 할 것이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [18]:
class QADataset(torch.utils.data.Dataset):
    def __init__ (self, data_dir: str, tokenizer, max_seq_len: int, mode = 'train', debug = False):
        self.mode = mode
        self.data = load_json(data_dir)

        # self.encodings = encodings
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.debug = debug
        self.pad_on_right = tokenizer.padding_side == "right"
        if mode == 'test':
            self.encodings, self.question_ids = self.preprocess()
        else:
            self.encodings, self.answers = self.preprocess()

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, index: int):
        print('index = ', index)
        return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}

    def preprocess(self):
        contexts, questions, answers, question_ids = self.read_squad()
        if self.mode == 'test':
            encodings = self.tokenizer(
                questions if self.pad_on_right else contexts,
                contexts if self.pad_on_right else questions,
                truncation=True, max_length = self.max_seq_len, padding=True)
            return encodings, question_ids
        else: # train or val
            self.add_end_idx(answers, contexts)
            encodings = self.tokenizer(
                questions if self.pad_on_right else contexts,
                contexts if self.pad_on_right else questions,
                truncation=True, max_length = self.max_seq_len, padding=True)
            self.add_token_positions(encodings, answers)

            return encodings, answers

    def read_squad(self):
        contexts = []
        questions = []
        question_ids = []
        answers = []

        # train - val split
        if self.mode == 'train':
            self.data['data'] = self.data['data'][:-1*int(len(self.data['data'])*0.1)]
        elif self.mode == 'val':
            self.data['data'] = self.data['data'][-1*int(len(self.data['data'])*0.1):]


        till = 100 if self.debug else len(self.data['data'])

        for group in self.data['data'][:till]:
            for passage in group['paragraphs']:
                context = passage['context']
                for qa in passage['qas']:
                    question = qa['question']
                    if self.mode == 'test':
                        contexts.append(context)
                        questions.append(question)
                        question_ids.append(qa['question_id'])
                    else: # train or val
                        for ans in qa['answers']:
                            contexts.append(context)
                            questions.append(question)

                            if qa['is_impossible']:
                                answers.append({'text':'','answer_start':-1})
                            else:
                                answers.append(ans)

        # return formatted data lists
        return contexts, questions, answers, question_ids

    def add_end_idx(self, answers, contexts):
        for answer, context in zip(answers, contexts):
            gold_text = answer['text']
            start_idx = answer['answer_start']
            end_idx = start_idx + len(gold_text)

            # in case the indices are off 1-2 idxs
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = end_idx
            else:
                for n in [1, 2]:
                    if context[start_idx-n:end_idx-n] == gold_text:
                        answer['answer_start'] = start_idx - n
                        answer['answer_end'] = end_idx - n
                    elif context[start_idx+n:end_idx+n] == gold_text:
                        answer['answer_start'] = start_idx + n
                        answer['answer_end'] = end_idx + n

    def add_token_positions(self, encodings, answers):
        # should use Fast tokenizer
        start_positions = []
        end_positions = []
        for i in range(len(answers)):
            if answers[i]['answer_start'] == -1:
                # set [CLS] token as answer if is_impossible
                start_positions.append(0)
                end_positions.append(1)
            else:
                start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))

                assert 'answer_end' in answers[i].keys(), f'no answer_end at {i}'
                end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

            # answer passage truncated
            if start_positions[-1] is None:
                start_positions[-1] = tokenizer.model_max_length
            # end position cannot be found, shift until found
            shift = 1
            while end_positions[-1] is None:
                end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
                shift += 1
        # char-based -> token based
        encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


In [122]:
class QADataset(torch.utils.data.Dataset):
    
    def __init__ (self, data_dir: str, tokenizer, max_seq_len: int, mode = 'train', debug = False):
        self.mode = mode
        self.data = load_json(data_dir)
        
        # self.encodings = encodings
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.debug = debug

        self.pad_on_right = tokenizer.padding_side == "right"

        if mode == 'test':
            self.encodings, self.question_ids = self.preprocess()
        else:
            self.encodings, self.answers = self.preprocess()
        
    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, index: int):
        return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}

    
    def preprocess(self):
        pad_on_right = self.pad_on_right
        doc_stride = 256
        contexts, questions, answers, question_ids = self.read_squad()
        self.contexts = contexts
        print('len contexts =', len(contexts))
        if self.mode == 'test':
            encodings = self.tokenizer(
                questions if pad_on_right else contexts,
                contexts if pad_on_right else questions,
                truncation="only_second" if pad_on_right else "only_first",
                max_length = self.max_seq_len,
                stride=doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length")
            return encodings, question_ids
        else: # train or val
            self.add_end_idx(answers, contexts)
            encodings = self.tokenizer(
                questions if pad_on_right else contexts,
                contexts if pad_on_right else questions,
                truncation="only_second" if pad_on_right else "only_first",
                max_length = self.max_seq_len,
                stride=doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length")
            self.add_token_positions(encodings, answers)
            for i in range(11):
                print(contexts[i][answers[i]['answer_start']:answers[i]['answer_end']])
        
            return encodings, answers
    
    def read_squad(self):
        contexts = []
        questions = []
        question_ids = []
        answers = []
        
        # train - val split
        if self.mode == 'train':
            self.data['data'] = self.data['data'][0:10]
            #self.data['data'] = self.data['data'][:-1*int(len(self.data['data'])*0.1)]
        elif self.mode == 'val':
            self.data['data'] = self.data['data'][-1*int(len(self.data['data'])*0.1):]
        print('count=',len(self.data['data']))
        
        till = 100 if self.debug else len(self.data['data'])
        
        for group in self.data['data'][:till]:
            for passage in group['paragraphs']:
                context = passage['context']
                for qa in passage['qas']:
                    question = qa['question']
                    if self.mode == 'test':
                        contexts.append(context)
                        questions.append(question)
                        question_ids.append(qa['question_id'])
                    else: # train or val
                        for ans in qa['answers']:
                            contexts.append(context)
                            questions.append(question)

                            if qa['is_impossible']:
                                answers.append({'text':'','answer_start':-1})
                            else:
                                answers.append(ans)
                
        # return formatted data lists
        return contexts, questions, answers, question_ids
    
    def add_end_idx(self, answers, contexts):
        for answer, context in zip(answers, contexts):
            gold_text = answer['text']
            start_idx = answer['answer_start']
            end_idx = start_idx + len(gold_text)

            # in case the indices are off 1-2 idxs
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = end_idx
            else:
                for n in [1, 2]:
                    if context[start_idx-n:end_idx-n] == gold_text:
                        answer['answer_start'] = start_idx - n
                        answer['answer_end'] = end_idx - n
                    elif context[start_idx+n:end_idx+n] == gold_text:
                        answer['answer_start'] = start_idx + n
                        answer['answer_end'] = end_idx + n

    def add_token_positions(self, tokenized_examples, answers):
        examples = self.data
        pad_on_right = self.pad_on_right
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # Let's label those examples!
        start_positions = []
        end_positions = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            if (i == 10):
                print('i, sample_index =', i, sample_index)
                print('offsets =', offsets)
                print('contexts[i]=', self.contexts[i])
                print('len(offsets)=', len(offsets))
                print(tokenizer.convert_ids_to_tokens(tokenized_examples.input_ids[10]))
                #for j in range(len(offsets)):
                    #print(self.contexts[i][offsets[j][0]:offsets[j][1]])
            #answers = examples["answers"][sample_index]
            # If no answers are given, set the cls_index as answer.
            #if len(answers[sample_index]["answer_start"]) == 0:
                #start_positions.append(cls_index)
                #end_positions.append(cls_index)
            #else:
            if True:
                # Start/end character index of the answer in the text.
                #start_char = answers[sample_index]["answer_start"][0]
                #end_char = start_char + len(answers["text"][0])
                start_char = answers[sample_index]["answer_start"]
                end_char = start_char + len(answers[sample_index]["text"])
                if i == 10: 
                    print('start_char =', start_char)
                    print('end_char =', end_char)
                    print(self.contexts[sample_index][start_char:end_char])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
                #print(sequence_ids)

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    start_positions.append(cls_index)
                    end_positions.append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    start_positions.append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    end_positions.append(token_end_index + 1)

        # char-based -> token based
        tokenized_examples.update({'start_positions': start_positions, 'end_positions': end_positions})

In [6]:
class QADataset(torch.utils.data.Dataset):
    
    def __init__ (self, data_dir: str, tokenizer, max_seq_len: int, mode = 'train', debug = False):
        self.mode = mode
        self.data = load_json(data_dir)
        
        # self.encodings = encodings
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.debug = debug

        self.pad_on_right = tokenizer.padding_side == "right"

        if mode == 'test':
            self.encodings, self.question_ids = self.preprocess()
        else:
            self.encodings, self.answers = self.preprocess()
        
    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, index: int):
        return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}

    
    def preprocess(self):
        pad_on_right = self.pad_on_right
        doc_stride = 256
        contexts, questions, answers, question_ids = self.read_squad()
        if self.mode == 'test':
            encodings = self.tokenizer(
                questions if pad_on_right else contexts,
                contexts if pad_on_right else questions,
                truncation="only_second" if pad_on_right else "only_first",
                max_length = self.max_seq_len,
                stride=doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length")
            return encodings, question_ids
        else: # train or val
            self.add_end_idx(answers, contexts)
            encodings = self.tokenizer(
                questions if pad_on_right else contexts,
                contexts if pad_on_right else questions,
                truncation="only_second" if pad_on_right else "only_first",
                max_length = self.max_seq_len,
                stride=doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length")
            self.add_token_positions(encodings, answers)
        
            return encodings, answers
    
    def read_squad(self):
        contexts = []
        questions = []
        question_ids = []
        answers = []
        
        # train - val split
        if self.mode == 'train':
            #self.data['data'] = self.data['data'][0:100]
            self.data['data'] = self.data['data'][:-1*int(len(self.data['data'])*0.1)]
        elif self.mode == 'val':
            self.data['data'] = self.data['data'][-1*int(len(self.data['data'])*0.1):]
        
        till = 100 if self.debug else len(self.data['data'])
        
        for group in self.data['data'][:till]:
            for passage in group['paragraphs']:
                context = passage['context']
                for qa in passage['qas']:
                    question = qa['question']
                    if self.mode == 'test':
                        contexts.append(context)
                        questions.append(question)
                        question_ids.append(qa['question_id'])
                    else: # train or val
                        for ans in qa['answers']:
                            contexts.append(context)
                            questions.append(question)

                            if qa['is_impossible']:
                                answers.append({'text':'','answer_start':-1})
                            else:
                                answers.append(ans)
                
        # return formatted data lists
        return contexts, questions, answers, question_ids
    
    def add_end_idx(self, answers, contexts):
        for answer, context in zip(answers, contexts):
            gold_text = answer['text']
            start_idx = answer['answer_start']
            end_idx = start_idx + len(gold_text)

            # in case the indices are off 1-2 idxs
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = end_idx
            else:
                for n in [1, 2]:
                    if context[start_idx-n:end_idx-n] == gold_text:
                        answer['answer_start'] = start_idx - n
                        answer['answer_end'] = end_idx - n
                    elif context[start_idx+n:end_idx+n] == gold_text:
                        answer['answer_start'] = start_idx + n
                        answer['answer_end'] = end_idx + n

    def add_token_positions(self, tokenized_examples, answers):
        examples = self.data
        pad_on_right = self.pad_on_right
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # Let's label those examples!
        start_positions = []
        end_positions = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            #answers = examples["answers"][sample_index]
            # If no answers are given, set the cls_index as answer.
            #if len(answers[sample_index]["answer_start"]) == 0:
                #start_positions.append(cls_index)
                #end_positions.append(cls_index)
            #else:
            if True:
                # Start/end character index of the answer in the text.
                #start_char = answers[sample_index]["answer_start"][0]
                #end_char = start_char + len(answers["text"][0])
                start_char = answers[sample_index]["answer_start"]
                end_char = start_char + len(answers[sample_index]["text"])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
                #print(sequence_ids)

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    start_positions.append(cls_index)
                    end_positions.append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    start_positions.append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    end_positions.append(token_end_index + 1)

        # char-based -> token based
        tokenized_examples.update({'start_positions': start_positions, 'end_positions': end_positions})

In [127]:
base_train_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'train.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'train')

In [88]:
base_train_dataset[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [125]:
for i in range(0,71):
    print(tokenizer.decode(base_train_dataset[i]['input_ids'][base_train_dataset[i]['start_positions']:
                                            base_train_dataset[i]['end_positions']]))
    print(tokenizer.decode(base_train_dataset[i]['input_ids'][base_train_dataset[i]['start_positions']:
                                            base_train_dataset[i]['end_positions']+1]))

기술경쟁
기술경쟁력
평균 32
평균 32위
종합과학기술력
종합과학기술력 평가
연공서열
연공서열제
산출지
산출지표
기술기획의 분권
기술기획의 분권화가
동태적
동태적 경쟁력
과정과
과정과 경로
국가기술혁신시스템이
국가기술혁신시스템이론
PPP frame
PPP framework

학습
과도한 순환보
과도한 순환보직
Port
Porter
표본이탈
표본이탈률
표본이탈
표본이탈 횟수
송헌
송헌재
심
심영상
이탈위험 표본관리방
이탈위험 표본관리방법
60
60세
표본이
표본이탈
패널데이
패널데이터
비례위험 회귀분석
비례위험 회귀분석 모형
동일표본 반복추적
동일표본 반복추적조사
해양공간계
해양공간계획
해양
해양공간
항거곤
항거곤란

강간
간음 － 남녀성기결합
간음 － 남녀성기결합설
항거곤
항거곤란
준강간
준강간죄
항거불
항거불능
5년
5년 이하
성적 자기결정권의
성적 자기결정권의 보호
항거불
항거불능
항거곤
항거곤란
간
간음
강간
강간죄
1992
1992년
물질적 경제적
물질적 경제적 자립
자립지원
자립지원 서비스
장기가출
장기가출자
청소년쉼
청소년쉼터
일시쉼
일시쉼터
2012
2012년
가출청
가출청소년
##40
##40시간

미국
유럽형사문제
유럽형사문제위원회
유럽평의회 사이버범죄협
유럽평의회 사이버범죄협약
디지털
디지털 데이터

노동
1985
1985년
실질
실질 금리
소비자물
소비자물가
Acemoglu and Guerrier
Acemoglu and Guerrieri
청소 및 경비용역직
청소 및 경비용역직 노동자
삶과
삶과환경
2006년 1
2006년 1월
음식물 폐기물 수거운반 대행
음식물 폐기물 수거운반 대행업체
59. 7
59. 7세
음식물 폐기물 수집
음식물 폐기물 수집 운반
프랑스 지역관리
프랑스 지역관리공사
최고가치 ( Best Value ) 입찰
최고가치 ( Best Value ) 입찰제
대행수수료 지급
대행수수료 지급방식
2012
2012년
사회적 양극화
사회적 양극화 문제
개인의 교육수
개인의 교육수준
인적자본투자이
인적자본투자이론
일반고등
일반고등학교


In [96]:
base_train_dataset[10]

{'input_ids': tensor([    2,  6436, 10707,  4292,  7961,  4279,  4034, 16734,  4112,  2702,
          4474,     3,    56, 11868,  9013,  4234,  6271,  4158,  4346,  4361,
          6392,  4234,  3667,  4112, 17563,  4105,  4239, 12452,  4880,  4176,
            18,  6421,  4129, 17563,  4105,  4034,  6379,    12, 13623, 29679,
            13,    16,  3794, 15883,    12,    52,  8296, 17372,    13,    16,
          2728, 10455,    12, 32526,  4078,    13,  2613,  6455,  7796,    18,
          6379,  4007,  4271,  6271,  4007,  3243,  4292,  6736,  4279,  4034,
         11529,  4110,  6455,  4279, 18781,  2279,  6274,  4234,  7006,  6671,
          4292, 11757,  7796,    18,  6421,  4129,  2279,  6274,  7006,  6671,
          4007,  4118,  3760,  4112,  7028,    19,  6839,    12,  7912, 11825,
         16384,  4019,  7047,    19, 23234, 10571, 33553,    13,    16, 10707,
          4162,    12,    86, 19430, 22304, 28604,  4019,  7047,    13,    16,
          7642,    12, 19190,  6515,  4

In [103]:
tokenizer('경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지')

{'input_ids': [2, 6516, 33216, 4292, 6303, 4279, 4219, 6516, 33216, 4292, 3092, 4304, 2967, 3249, 4034, 6472, 4292, 6303, 4279, 4031, 6628, 6605, 8986, 4151, 11910, 3758, 6303, 4034, 2702, 4200, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [83]:
tokenizer.decode(base_train_dataset.encodings.input_ids[0])

'[CLS] 경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지 [SEP] 이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재의 경쟁력보다는 미래의 경쟁력을 평가하도록 평가의 대상을 확대해야 한다. 지금까지의 경쟁력 평가가 현위치를 중심으로 한 것이었다면, 앞으로의 대안은 과정과 경로를 중시하는 것이 되어야 한다는 것이다. 사실 지식정보화사회에서 가진 것이란 허망한 것이다. 현재의 기술수준이 높더라고, 이를 끊임없이 개량하고 새로운 기술을 발전시킬 수 있는 능력이 없다면 미래의 전망은 어둡다. 기술이나 정보는 실물 자산보다 진부화율이 매우 높다. 둘째, 기술경쟁력 평가를 순위를 매기는 작업이 아니라 이를 통하여 장점과 단점을 파악하는 과정으로 활용하여야 한다. 순위는 대중적인 흥미는 끌 수 있지만, 그것으로부터 교훈이 도출되는 것은 아니다. 기술경쟁력의 평가를 통해서 외국시스템의 장점을 배우고, 한국 시스템의 단점을 교정하는 대안이 발견될 수 있다. 경쟁이란 남과 비교하는 것을 의미한다. 내가 아무리 잘 하더라도 남이 나보다 더 잘 한다면 경쟁에서 진다. 그래서 경쟁은 각박하지만, 과학기술은 경쟁을 피할 수 있는 분야가 아니다. 경쟁 상대국을 연구하고, 경쟁 상대국을 앞설 수 있는 방안을 강구하기 위해서 기술경쟁력에 관한 연구가 더욱 활발해져야 할 것이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [58]:
base_train_dataset[0]

{'input_ids': tensor([    2,  6516, 33216,  4292,  6303,  4279,  4219,  6516, 33216,  4292,
          3092,  4304,  2967,  3249,  4034,  6472,  4292,  6303,  4279,  4031,
          6628,  6605,  8986,  4151, 11910,  3758,  6303,  4034,  2702,  4200,
             3,  3240,  2129,  4073, 28893,  7200,  4283,  6344,  4158,  4346,
          4361,  6392,  6472,  4234,  7511,  4112,  2419,  6274,  4239, 12452,
          4880,  4176,    18,  9270,    16,  6339,  4234,  7685,  4275,  6913,
          6706,  4234,  7685,  4292,  6392,  4279, 26147,  6392,  4234,  6391,
          4292,  6626, 18991,  6217,    18,  6292,  4149,  4200,  4234,  7685,
          6392,  4070,  3794, 15883,  4110,  6593, 10749,  3757,  2048,  4007,
          4480,  7599,    16,  3092, 10749,  4234,  8560,  4112,  6379,  4047,
         10455,  4110, 11160,  4279,  4034,  2048,  4007,  2411, 16868,  6403,
          2048, 24387,    18,  6278,  7260, 21590,  4162, 32459,  4073,  4129,
          6971,  2048,  4007,  4271, 27

In [7]:
base_train_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'train.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'train')
base_val_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'train.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'val')

In [9]:
tokenizer.decode(base_train_dataset[0]['input_ids'])

'[CLS] 경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지 [SEP] 이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재의 경쟁력보다는 미래의 경쟁력을 평가하도록 평가의 대상을 확대해야 한다. 지금까지의 경쟁력 평가가 현위치를 중심으로 한 것이었다면, 앞으로의 대안은 과정과 경로를 중시하는 것이 되어야 한다는 것이다. 사실 지식정보화사회에서 가진 것이란 허망한 것이다. 현재의 기술수준이 높더라고, 이를 끊임없이 개량하고 새로운 기술을 발전시킬 수 있는 능력이 없다면 미래의 전망은 어둡다. 기술이나 정보는 실물 자산보다 진부화율이 매우 높다. 둘째, 기술경쟁력 평가를 순위를 매기는 작업이 아니라 이를 통하여 장점과 단점을 파악하는 과정으로 활용하여야 한다. 순위는 대중적인 흥미는 끌 수 있지만, 그것으로부터 교훈이 도출되는 것은 아니다. 기술경쟁력의 평가를 통해서 외국시스템의 장점을 배우고, 한국 시스템의 단점을 교정하는 대안이 발견될 수 있다. 경쟁이란 남과 비교하는 것을 의미한다. 내가 아무리 잘 하더라도 남이 나보다 더 잘 한다면 경쟁에서 진다. 그래서 경쟁은 각박하지만, 과학기술은 경쟁을 피할 수 있는 분야가 아니다. 경쟁 상대국을 연구하고, 경쟁 상대국을 앞설 수 있는 방안을 강구하기 위해서 기술경쟁력에 관한 연구가 더욱 활발해져야 할 것이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

**순서가 context, question임.  why?**

In [17]:
for key,val in base_val_dataset.encodings.items():
    print(key)
    print(val[0])
    break

input_ids
[2, 20967, 4801, 4162, 6940, 2728, 9994, 4028, 6891, 4292, 6260, 6325, 8188, 4007, 7370, 4162, 2411, 4181, 4129, 6570, 4007, 17051, 4279, 4219, 3249, 4025, 3, 6721, 10749, 6377, 12178, 4225, 4142, 6371, 4073, 6985, 6700, 8767, 4110, 7083, 4279, 4737, 4176, 18, 6417, 4283, 6377, 12178, 4225, 4142, 6371, 4112, 6417, 4283, 10368, 4292, 6442, 4114, 5180, 2967, 3249, 18781, 16, 29676, 4034, 3240, 3330, 7002, 4414, 4162, 4239, 7713, 6325, 4147, 4297, 4086, 7235, 16, 2089, 4332, 17450, 4047, 6377, 24105, 6448, 4234, 7601, 4292, 7083, 4279, 4737, 4176, 18, 7233, 4556, 10008, 4234, 7002, 4199, 4195, 4034, 11663, 4234, 28, 18, 29, 9, 2575, 6377, 4120, 4031, 6766, 4139, 21, 18, 22, 9, 3238, 27, 18, 24, 4238, 6502, 4292, 6370, 4219, 3249, 4025, 7605, 7002, 4199, 4195, 4110, 13177, 6995, 4114, 4343, 4226, 4569, 4070, 6700, 10008, 4234, 8767, 4070, 2413, 2048, 10749, 6612, 4880, 4176, 18, 9994, 4028, 6891, 2728, 20967, 4801, 4162, 6940, 4110, 6260, 6325, 8188, 4007, 7370, 4162, 4479, 4181,

In [58]:

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 25262
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2762
    })
})

In [8]:
#train_dataset = tokenized_datasets['train']
#val_dataset = tokenized_datasets['validation']
train_dataset = base_train_dataset
val_dataset = base_val_dataset

In [9]:
# DataLoader
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=config['DATALOADER']['batch_size'],
                              num_workers=config['DATALOADER']['num_workers'],
                              shuffle=config['DATALOADER']['shuffle'],
                              pin_memory=config['DATALOADER']['pin_memory'],
                              drop_last=config['DATALOADER']['drop_last'])
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=config['DATALOADER']['batch_size'],
                            num_workers=config['DATALOADER']['num_workers'],
                            shuffle=False,
                            pin_memory=config['DATALOADER']['pin_memory'],
                            drop_last=config['DATALOADER']['drop_last'])

logger.info(f"Load data, train:{len(train_dataset)} val:{len(val_dataset)}")


In [10]:
config['TRAINER']['learning_rate']

5e-05

In [15]:
"""
02. Set model
"""
# Load model
model_name = config['TRAINER']['model']
model = get_model(model_name=model_name, pretrained=config['TRAINER']['pretrained']).to(device)

checkpoint = torch.load(os.path.join('results/train/20220611_232924', 'model.pt'))
model.load_state_dict(checkpoint['model'])

#for param in model.model.electra.parameters():
#    param.requires_grad = False

"""
03. Set trainer
"""
# Optimizer
optimizer = get_optimizer(optimizer_name=config['TRAINER']['optimizer'])

# Prepare optimizer and schedule (linear warmup and decay)
weight_decay = 0.1
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
#optimizer = optimizer(params=model.parameters(),lr=config['TRAINER']['learning_rate'])
optimizer = optimizer(optimizer_grouped_parameters,lr=config['TRAINER']['learning_rate'])
#optimizer = optimizer(params=model.parameters(),lr=5e-4)

# lr reducer
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.800, verbose=True)

# Loss
loss = get_loss(loss_name=config['TRAINER']['loss'])

# Metric
metrics = {metric_name: get_metric(metric_name) for metric_name in config['TRAINER']['metric']}

# Early stoppper
early_stopper = EarlyStopper(patience=config['TRAINER']['early_stopping_patience'],
                            mode=config['TRAINER']['early_stopping_mode'],
                            logger=logger)
# AMP
if config['TRAINER']['amp'] == True:
    from apex import amp
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

# Trainer
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  scheduler=scheduler,
                  loss=loss,
                  metrics=metrics,
                  device=device,
                  logger=logger,
                  tokenizer=tokenizer,
                  amp=amp if config['TRAINER']['amp'] else None,
                  interval=config['LOGGER']['logging_interval'])
"""
Logger
"""
# Recorder
recorder = Recorder(record_dir=RECORDER_DIR,
                    model=model,
                    optimizer=optimizer,
                    scheduler=None,
                    amp=amp if config['TRAINER']['amp'] else None,
                    logger=logger)

# !Wandb
if config['LOGGER']['wandb'] == True:
    wandb_project_serial = 'template'
    wandb_username =  ''
    wandb.init(project=wandb_project_serial, dir=RECORDER_DIR, entity=wandb_username)
    wandb.run.name = train_serial
    wandb.config.update(config)
    wandb.watch(model)

# Save train config
save_yaml(os.path.join(RECORDER_DIR, 'train_config.yml'), config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.bias', 

Adjusting learning rate of group 0 to 5.0000e-05.
Adjusting learning rate of group 1 to 5.0000e-05.


In [46]:
for param in model.model.electra.parameters():
    param.requires_grad = False

In [14]:
it = iter(train_dataloader)

In [15]:
inputs = next(it)

In [16]:
inputs

{'input_ids': tensor([[    2,  6358,  7353,  ...,     0,     0,     0],
         [    2,  7612,  4250,  ...,     0,     0,     0],
         [    2,  7644,  7591,  ...,     0,     0,     0],
         ...,
         [    2,  8869,  8327,  ...,     0,     0,     0],
         [    2,  7644,  6244,  ...,     0,     0,     0],
         [    2, 21766,  4234,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'start_positions': tensor([116,  37, 244,  33,  55, 146, 162,  44, 117, 200, 142,  26, 275,  52,
          23,  42]),
 'end_posi

In [18]:
device = torch.device("cuda:0")
input_ids = inputs['input_ids'].to(device)
attention_mask=inputs['attention_mask'].to(device)
start_positions=inputs['start_positions'].to(device)
end_positions=inputs['end_positions'].to(device)

In [20]:
outputs = model(input_ids,
                attention_mask,
               start_positions,
               end_positions)

In [22]:
loss, start_logits, end_logits = outputs

In [25]:
outputs['loss']

tensor(2.2061, device='cuda:0', grad_fn=<DivBackward0>)

In [31]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [32]:
start_logits

tensor([[ -7.7879, -12.3271, -12.1101,  ..., -12.6438, -12.6048,  -7.7877],
        [-10.0337, -11.5036, -12.5624,  ..., -12.5828, -12.5603, -10.0339],
        [-10.4607, -12.3261, -12.3840,  ..., -12.6758, -12.6417, -10.4603],
        ...,
        [-10.7596, -12.2284, -12.5344,  ..., -12.7009, -12.6098, -10.7596],
        [-10.9786, -12.0141, -12.0707,  ..., -12.7997, -12.7316, -10.9787],
        [ -9.8129, -12.2328, -12.3911,  ..., -12.7641, -12.6870,  -9.8129]],
       device='cuda:0', grad_fn=<CopyBackwards>)

In [33]:
start_idx = torch.argmax(start_logits, dim=1).cpu().tolist() 
end_idx = torch.argmax(end_logits, dim=1).cpu().tolist()

In [34]:
start_idx

[37, 37, 244, 33, 55, 146, 162, 44, 117, 196, 142, 26, 275, 31, 23, 42]

In [35]:
end_idx

[41, 40, 244, 33, 57, 147, 164, 53, 124, 200, 147, 31, 278, 35, 25, 46]

In [36]:
start_positions

tensor([116,  37, 244,  33,  55, 146, 162,  44, 117, 200, 142,  26, 275,  52,
         23,  42], device='cuda:0')

In [37]:
end_positions

tensor([117,  40, 244,  33,  57, 148, 164,  53, 124, 200, 147,  31, 277,  56,
         25,  46], device='cuda:0')

In [43]:
for i in range(16):
    print(tokenizer.decode(input_ids[i][start_idx[i]:end_idx[i]+1]))
    #print(tokenizer.decode(input_ids[i][start_positions[i]:end_positions[i]+1]))

직접고용 비정규직 근로자
스토커규제법
고령자
호주
인적자본
신재준
표준오차
Guggemos and Horvath
교육부와 시도교육청간 권한 배분
해상에 있는 선박
확정기간혼합방식
공익신고자보호법
기타 전공 영역 전공
국가통계시스템
송헌재
장애인 의무고용률


In [ ]:
"""
04. TRAIN
"""
# Train
n_epochs = config['TRAINER']['n_epochs']
for epoch_index in range(n_epochs):

    # Set Recorder row
    row_dict = dict()
    row_dict['epoch_index'] = epoch_index
    row_dict['train_serial'] = train_serial

    """
    Train
    """
    print(f"Train {epoch_index}/{n_epochs}")
    logger.info(f"--Train {epoch_index}/{n_epochs}")
    trainer.train(dataloader=train_dataloader, epoch_index=epoch_index, tokenizer=tokenizer, mode='train')

    row_dict['train_loss'] = trainer.loss_mean
    row_dict['train_elapsed_time'] = trainer.elapsed_time

    for metric_str, score in trainer.score_dict.items():
        row_dict[f"train_{metric_str}"] = score
    trainer.clear_history()

    """
    Validation
    """
    print(f"Val {epoch_index}/{n_epochs}")
    logger.info(f"--Val {epoch_index}/{n_epochs}")
    trainer.train(dataloader=val_dataloader, epoch_index=epoch_index, tokenizer=tokenizer, mode='val')

    row_dict['val_loss'] = trainer.loss_mean
    row_dict['val_elapsed_time'] = trainer.elapsed_time 

    for metric_str, score in trainer.score_dict.items():
        row_dict[f"val_{metric_str}"] = score
    trainer.clear_history()

    """
    Record
    """
    recorder.add_row(row_dict)
    recorder.save_plot(config['LOGGER']['plot'])

    #!WANDB
    if config['LOGGER']['wandb'] == True:
        wandb.log(row_dict)

    """
    Early stopper
    """
    early_stopping_target = config['TRAINER']['early_stopping_target']
    early_stopper.check_early_stopping(loss=row_dict[early_stopping_target])

    if early_stopper.patience_counter == 0:
        recorder.save_weight(epoch=epoch_index)
        best_row_dict = copy.deepcopy(row_dict)

    if early_stopper.stop == True:
        logger.info(f"Early stopped, counter {early_stopper.patience_counter}/{config['TRAINER']['early_stopping_patience']}")

        if config['LOGGER']['wandb'] == True:
            wandb.log(best_row_dict)
        break

Train 0/20


  7%|█████▌                                                                              | 104/1579 [00:29<07:02,  3.49it/s]

In [37]:
model.model.electra

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(35000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0): ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((256,), e

In [38]:
for param in model.model.electra.parameters():
    param.requires_grad = False

In [32]:
model.model.electra

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(35000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0): ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((256,), e

In [13]:
for n, p in model.named_parameters():
    print(n)

model.electra.embeddings.word_embeddings.weight
model.electra.embeddings.position_embeddings.weight
model.electra.embeddings.token_type_embeddings.weight
model.electra.embeddings.LayerNorm.weight
model.electra.embeddings.LayerNorm.bias
model.electra.encoder.layer.0.attention.self.query.weight
model.electra.encoder.layer.0.attention.self.query.bias
model.electra.encoder.layer.0.attention.self.key.weight
model.electra.encoder.layer.0.attention.self.key.bias
model.electra.encoder.layer.0.attention.self.value.weight
model.electra.encoder.layer.0.attention.self.value.bias
model.electra.encoder.layer.0.attention.output.dense.weight
model.electra.encoder.layer.0.attention.output.dense.bias
model.electra.encoder.layer.0.attention.output.LayerNorm.weight
model.electra.encoder.layer.0.attention.output.LayerNorm.bias
model.electra.encoder.layer.0.intermediate.dense.weight
model.electra.encoder.layer.0.intermediate.dense.bias
model.electra.encoder.layer.0.output.dense.weight
model.electra.encoder.